In [1]:
import os 
from glob import glob

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline

In [2]:
train_dir = 'mnist_png/mnist_png/training/'
test_dir = 'mnist_png/mnist_png/testing/'

In [3]:
batch_size = 32
num_classes = 10
epochs = 1

learning_rate = 0.001

In [4]:
def build_model(input_shape):
    inputs = layers.Input(input_shape)
    net = layers.Conv2D(32, (3, 3), padding='same')(inputs)
    net = layers.Activation('relu')(net)
    net = layers.Conv2D(32, (3, 3))(net)
    net = layers.Activation('relu')(net)
    net = layers.MaxPooling2D(pool_size=(2, 2))(net)
    net = layers.Dropout(0.25)(net)
    
    net = layers.Conv2D(64, (3, 3), padding='same')(net)
    net = layers.Activation('relu')(net)
    net = layers.Conv2D(64, (3, 3))(net)
    net = layers.Activation('relu')(net)
    net = layers.MaxPooling2D(pool_size=(2, 2))(net)
    net = layers.Dropout(0.25)(net)

    net = layers.Flatten()(net)
    net = layers.Dense(512)(net)
    net = layers.Activation('relu')(net)
    net = layers.Dropout(0.5)(net)
    net = layers.Dense(num_classes)(net)
    net = layers.Activation('softmax')(net)

    return tf.keras.Model(inputs=inputs, outputs=net)

model = build_model((28, 28, 1))
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate),
              metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [5]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(28, 28),
    color_mode='grayscale',
    batch_size=32,
    class_mode='categorical',
    shuffle=True,
#     save_to_dir='out_images'
)

validation_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(28, 28),
    color_mode='grayscale',
    batch_size=32,
    class_mode='categorical')

Found 60000 images belonging to 10 classes.
Found 10000 images belonging to 10 classes.


In [6]:
model.fit_generator(
        train_generator,
        steps_per_epoch=2000,
        epochs=1,
        validation_data=validation_generator,
        validation_steps=800
)

Instructions for updating:
Use tf.cast instead.
1875/1875 [==============================] - 567s 302ms/step - loss: 0.2874 - acc: 0.9076 - val_loss: 0.0597 - val_acc: 0.9802


# Get h5

In [8]:
# Optimizer부터 Model과 Weight까지 모두 저장. Parameter로 Optimizer 저장을 제외 가능. 
# Opimizer도 저장하면 Model Load와 함께 바로 학습도 가능
model.save('saved_model/hdf5/mnist_cnn.h5')

# Weight만 저장. Model은 다시 짜야 하지만, Layer 층들에 대해 디테일한 Customize 가능. 
model.save_weights('saved_model/hdf5/weights/mnist_cnn')


Consider using a TensorFlow optimizer from `tf.train`.
Instructions for updating:
Use tf.train.CheckpointManager to manage checkpoints rather than manually editing the Checkpoint proto.


In [9]:
model = tf.keras.models.load_model('saved_model/hdf5/mnist_cnn.h5')

In [ ]:
# Train Loaded Model
model.fit_generator(
        train_generator,
        steps_per_epoch=2000,
        epochs=1,
        validation_data=validation_generator,
        validation_steps=800
)

## Save PB

https://medium.com/@alexkn15/tensorflow-save-model-for-use-in-java-or-c-ab351a708ee4  
https://dev-strender.github.io/articles/2017-10/tensorflow-load-model-with-cpp  

위의 링크에서 Jave 또는 C++로 embedding 하는 방법 참고  
첫번째 PB 파일은 모델과 weight를 따로 저장하지만, 두번째 방법은 하나의 PB에 모델과 weight 둘 다 한번에 저장

In [ ]:
import tensorflow.keras.backend as K

import tensorflow.python.saved_model
from tensorflow.python import saved_model
from tensorflow.python.saved_model import tag_constants
from tensorflow.python.saved_model.signature_def_utils_impl import predict_signature_def

In [ ]:
sess = K.get_session()

save_folder = 'saved_model/pb/'

input_name = model.input.name  # Embedding을 받을 개발자가 input과 output name을 요구
output_name = model.output.name

print('Input 이름:', input_name)
print('Output 이름:', output_name)

In [ ]:
builder = saved_model.builder.SavedModelBuilder(save_folder)
signature = predict_signature_def(inputs={model.input_names[0]: model.input},  # {인풋 이름: 인풋 레이어}
                                  outputs={model.output_names[0]: model.output})
# using custom tag instead of: tags=[tag_constants.SERVING]
builder.add_meta_graph_and_variables(sess=sess,
                                     tags=["serve"],
                                     signature_def_map={'predict': signature})
builder.save()

## Save PB - 2

In [ ]:
def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    """
    Freezes the state of a session into a pruned computation graph.

    Creates a new computation graph where variable nodes are replaced by
    constants taking their current value in the session. The new graph will be
    pruned so subgraphs that are not necessary to compute the requested
    outputs are removed.
    @param session The TensorFlow session to be frozen.
    @param keep_var_names A list of variable names that should not be frozen,
                          or None to freeze all the variables in the graph.
    @param output_names Names of the relevant graph outputs.
    @param clear_devices Remove the device directives from the graph for better portability.
    @return The frozen graph definition.
    """
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = tf.compat.v1.graph_util.convert_variables_to_constants(
            session, input_graph_def, output_names, freeze_var_names)
        return frozen_graph

In [ ]:
# 무엇 무엇을 Froze 했다고 하면 성공
frozen_graph = freeze_session(K.get_session(),
                              output_names=[out.op.name for out in model.outputs])

In [ ]:
tf.train.write_graph(frozen_graph, "saved_model/frozen_pb/", "saved_model.pb", as_text=False)

## Load PB File

In [ ]:
from PIL import Image

image = np.array(Image.open('mnist_png/mnist_png/testing/0/3.png'))

In [ ]:
with tf.Session(graph=tf.Graph()) as sess:
    tf.saved_model.loader.load(sess, ["serve"], save_folder)  # [tf.saved_model.tag_constants.SERVING] 
    graph = tf.get_default_graph()

#     X = tf.placeholder(tf.float32, (1, 28, 28, 1), name='X')
    pred = sess.run(output_name, feed_dict={input_name: image.reshape(1, 28, 28, 1)})

# Convert Keras to ONNX

onnx는 caffe나 pytorch까지 호환이 되도록 해주는 저장 형식  
(... 현재 디버깅 중)

In [ ]:
# pip install keras2onnx

In [10]:
import keras2onnx

onnx_model = keras2onnx.convert_keras(model, model.name)

Using TensorFlow backend.


Instructions for updating:
Use tf.compat.v1.graph_util.remove_training_nodes


FailedPreconditionError: Error while reading resource variable dense_1_1/bias from Container: localhost. This could mean that the variable was uninitialized. Not found: Container localhost does not exist. (Could not find resource: localhost/dense_1_1/bias)
	 [[node dense_1_1/bias/Read/ReadVariableOp (defined at <ipython-input-9-5e3c8bf0e0ac>:1) ]]

Caused by op 'dense_1_1/bias/Read/ReadVariableOp', defined at:
  File "C:\Users\June\AppData\Local\Continuum\anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\June\AppData\Local\Continuum\anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\June\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\June\AppData\Local\Continuum\anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\June\AppData\Loc